In [1]:
import pandas as pd
from keras.callbacks import History, ReduceLROnPlateau,EarlyStopping,ModelCheckpoint
import os
import numpy as np
from data_analysis import calculate_metrics, load_weights_and_evaluate
from model_builders import GCN_pretraining
from hyperparameter_tuning_GCN import objective
from functools import partial
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
import pickle
import dill
from hyper_mining import objective_fn

Using TensorFlow backend.
C:\Users\tomas\miniconda3\envs\binding\lib\site-packages\tensorflow\python\framework\dtypes.py:523: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\tomas\miniconda3\envs\binding\lib\site-packages\tensorflow\python\framework\dtypes.py:524: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\tomas\miniconda3\envs\binding\lib\site-packages\tensorflow\python\framework\dtypes.py:525: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\tomas\miniconda3\envs\binding

AttributeError: module 'keras_applications' has no attribute 'set_keras_submodules'

In [2]:
fspace = {
    'conv1' : hp.quniform('conv1', 32, 64, 8),
    'conv2' : hp.quniform('conv2', 64, 128, 8),
    'conv3' : hp.quniform('conv3', 128, 168, 8),
    'fp' : hp.quniform('fp', 96, 196, 8),
    'dense1' : hp.quniform('dense1',96,512,32),
    'dense2' : hp.quniform('dense2',96,512,32),
    'dense3' : hp.quniform('dense3',64,512,32),
    'dropout_rate' : hp.uniform('dropout_rate',0.1,0.5),
    'lr' : hp.uniform('lr',0.00001,0.01),
    'n_epochs' : hp.quniform('n_epochs',15,60,5),
    'colsample_bylevel' : hp.uniform('colsample_bylevel', 0.1, 1), 
    'colsample_bytree' : hp.uniform('colsample_bytree', 0.1, 1), 
    'gamma' : hp.uniform('gamma', 0.1, 1), 
    'learning_rate' : hp.uniform('learning_rate', 0.1, 1),
    'max_delta_step' : hp.quniform('max_delta_step',1,10,1),
    'max_depth' : hp.quniform('max_depth',6, 12, 1),
    'min_child_weight' : hp.quniform('min_child_weight',10 ,500 ,5),
    'reg_alpha' : hp.uniform('reg_alpha',0.1,100),
    'reg_lambda' : hp.uniform('reg_lambda',0.1,100),
    'subsample' : hp.uniform('subsample',0.1,1.0),
    'max_bin' : hp.quniform('max_bin',16,256,16),
    'margin' : hp.uniform('margin',0.2,2)
}

In [3]:
target = 'p38'
base_path = f'C:/Users/tomas/Documents/GitHub/kinase_binding'

data_fpath = base_path+f'/data/{target}/data.csv'
df=pd.read_csv(data_fpath).set_index('biolab_index')

with open(base_path+f'/data/{target}/train_val_folds.pkl', "rb") as in_f:
    train_val_folds = dill.load(in_f)
with open(base_path+f'/data/{target}/train_test_folds.pkl', "rb") as in_f:
    train_test_folds = dill.load(in_f)

In [4]:
training_list = [df.loc[train_val_folds[0][0]],
                 df.loc[train_val_folds[1][0]],
                 df.loc[train_val_folds[2][0]],
                 df.loc[train_val_folds[3][0]],
                 df.loc[train_val_folds[4][0]],
                 df.loc[train_val_folds[5][0]],
                 ]
validation_list = [df.loc[train_val_folds[0][1]],
                   df.loc[train_val_folds[1][1]],
                   df.loc[train_val_folds[2][1]],
                   df.loc[train_val_folds[3][1]],
                   df.loc[train_val_folds[4][1]],
                   df.loc[train_val_folds[5][1]],
                   ]

In [5]:
fmin_objective = partial(objective, train_sets = training_list, val_sets = validation_list)

In [6]:
def run_trials():

    trials_step = 2  # how many additional trials to do after loading saved trials. 1 = save after iteration
    max_trials = 3  # initial max_trials. put something small to not have to wait

    
    try:  # try to load an already saved trials object, and increase the max
        trials = pickle.load(open("gcn_xgb.hyperopt", "rb"))
        print("Found saved Trials! Loading...")
        max_trials = len(trials.trials) + trials_step
        print("Rerunning from {} trials to {} (+{}) trials".format(len(trials.trials), max_trials, trials_step))
    except:  # create a new trials object and start searching
        trials = Trials()

    best = fmin(fn = fmin_objective, space = fspace, algo=tpe.suggest, max_evals=max_trials, trials=trials)

    print("Best:", best)
    
    # save the trials object
    with open("gcn_xgb.hyperopt", "wb") as f:
        pickle.dump(trials, f)
    return(trials)

In [7]:
trials = run_trials()

Found saved Trials! Loading...
Rerunning from 4 trials to 6 (+2) trials
LAYER 0                                                                                                                
LAYER 1                                                                                                                
LAYER 2                                                                                                                
 67%|█████████████████████████████████████████████▎                      | 4/6 [00:05<00:02,  1.41s/trial, best loss=?]

C:\Users\tomas\miniconda3\envs\binding\lib\site-packages\tensorflow\python\ops\gradients_impl.py:112: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "



LAYER 0                                                                                                                
LAYER 1                                                                                                                
LAYER 2                                                                                                                
LAYER 0                                                                                                                
LAYER 1                                                                                                                
LAYER 2                                                                                                                
LAYER 0                                                                                                                
LAYER 1                                                                                                                
LAYER 2                                 

In [8]:
from numba import cuda 
device = cuda.get_current_device()
device.reset()